# IMU Data Analysis

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.io import load_long_format_csv
from biopsykit.stats import StatsPipeline
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.data_cleaning.imu import clean_statistical_outlier
from carwatch_analysis.stats import create_unique_night_id, stats_pipeline_imu_features
from carwatch_analysis.plotting import boxplot_imu_features

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette}
theme_kwargs_scale = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 1.25}

tight_layout_params_boxplot = dict(rect=(0, 0, 0.85, 1.0), pad=0.2)

sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
plot_path = result_path.joinpath("plots")
stats_path = result_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([result_path, plot_path, stats_path])

## Import

In [ ]:
imu_data = load_long_format_csv(export_path.joinpath("imu_static_moment_features_cleaned.csv"))
imu_data.head()

In [ ]:
imu_data.index.get_level_values("imu_feature").unique()

## Data Selection

In [ ]:
selected_features = ["sm_max_position", "sm_max_60", "sm_number_60", "sm_mean_60", "sm_std_60"]

imu_features = imu_data.xs(("selfreport", "last_30min"), level=("wakeup_type", "time_span"))
imu_features = multi_xs(imu_features, feature_cols, level="imu_feature")

imu_features.head()

## Data Cleaning

In [ ]:
print(f"Number of Nights before cleaning: {imu_features.unstack(['imu_feature']).shape[0]}")

### Remove Statistical Outlier ($> 3 \sigma$)

Remove nights where features differ more than 3 standard deviations from the mean

In [ ]:
imu_features = clean_statistical_outlier(imu_features)
imu_features.head()

## Plots and Statistics

### Condition

#### Class Distribution

In [ ]:
describe_groups_df(imu_features, "condition")

#### Statistics

In [ ]:
variable = "condition"

pipeline = stats_pipeline_imu_features(imu_features, variable)

pipeline.export_statistics(stats_path.joinpath(f"stats_imu_features_{variable}.xlsx"))
pipeline.display_results(prep=True, sig_only={"test": False, "posthoc": True})

#### Plots

In [ ]:
boxplot_imu_features(imu_features, "condition", "sm_std_60", pipeline)

In [ ]:
boxplot_imu_features(imu_features, "condition", "sm_mean_60", pipeline)

### Pairplots

In [ ]:
df_pairplot = imu_features["data"].unstack("imu_feature").reset_index(level="condition")

g = sns.pairplot(data=df_pairplot, hue="condition", corner=True)
g = g.map_lower(sns.kdeplot, levels=3, color=".2", alpha=0.5)